# Preprocesamiento

## Importación de librerías

In [2]:
import pandas as pd
import numpy as np

## Lectura de datos

In [4]:
df = pd.read_csv("../../data/raw/historico_completo.csv")
df_est = pd.read_csv("../../data/raw/estaciones.csv")
df_integrado = pd.merge(df, df_est, on=["codigoEstacion", "provincia_id"])
df_integrado.head()

,fecha,dia,tempMedia,tempMax,horMinTempMax,tempMin,horMinTempMin,humedadMedia,humedadMax,horMinHumMax,...,bajoplastico,activa,visible,longitud,latitud,altitud,xutm,yutm,huso,provincia_nombre
0,2005-01-01,1,9.07,18.58,14:30,0.658,07:20,67.31,87.2,07:40,...,False,True,True,060102000W,364525000N,39,230650.0,4072170.0,30,Cádiz
1,2005-01-02,2,9.02,12.17,14:50,5.810,21:30,84.70,91.8,23:30,...,False,True,True,060102000W,364525000N,39,230650.0,4072170.0,30,Cádiz
2,2005-01-03,3,8.18,15.91,14:20,3.196,06:10,80.60,92.9,11:50,...,False,True,True,060102000W,364525000N,39,230650.0,4072170.0,30,Cádiz
3,2005-01-04,4,10.55,18.26,12:40,3.608,07:30,68.63,87.2,06:50,...,False,True,True,060102000W,364525000N,39,230650.0,4072170.0,30,Cádiz
4,2005-01-05,5,9.82,17.78,14:50,2.802,07:30,67.62,87.5,06:10,...,False,True,True,060102000W,364525000N,39,230650.0,4072170.0,30,Cádiz


In [6]:
df_integrado.to_csv("../../data/processed/datos_integrados.csv", sep=";", index=False)